In [ ]:
from trl import GRPOTrainer, GRPOConfig
from datasets import load_dataset

# 1. Load your dataset
dataset = load_dataset("trl-lib/tldr", split="train")


# 2. Define a simple reward function
def reward_func(completions, **kwargs):
    """Example: Reward longer completions"""
    return [float(len(completion)) for completion in completions]


# 3. Configure training
training_args = GRPOConfig(
    output_dir="output",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    logging_steps=10,
    learning_rate=2e-5,
    weight_decay=0.01,
    max_grad_norm=1.0,
    warmup_steps=100,
    lr_scheduler_type="cosine_with_restarts",
    fp16=True,
    bf16=False,
    optim="adamw_torch",
    max_steps=1000,
    save_steps=100,
    save_total_limit=2,
    evaluation_strategy="steps",
    eval_steps=100,
    eval_accumulation_steps=1,
    use_flash_attention_2=True,
    use_fast_tokenizer=True,
    use_peft=True,
    use_lora=True,
    use_gradient_checkpointing=True,
    use_torch_compile=True,
    use_torch_profiler=True,
    use_vllm=True,
    use_bitsandbytes=True,
    use_peft=True,
    use_lora=True,
    use_gradient_checkpointing=True,
    use_torch_compile=True,
    use_torch_profiler=True,
    
)

# 4. Initialize and train
trainer = GRPOTrainer(
    model="Qwen/Qwen2-0.5B-Instruct",  # e.g. "Qwen/Qwen2-0.5B-Instruct"
    args=training_args,
    train_dataset=dataset,
    reward_funcs=reward_func,
)
trainer.train()

Generating test split: 100%|██████████| 6553/6553 [00:00<00:00, 349449.79 examples/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 11.79 GiB. GPU 0 has a total capacity of 7.75 GiB of which 3.39 GiB is free. Including non-PyTorch memory, this process has 4.31 GiB memory in use. Of the allocated memory 3.76 GiB is allocated by PyTorch, and 405.64 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)